In [82]:
import pandas as pd
from collections import Counter
import numpy as np
from copy import deepcopy
import random
from functools import reduce
from itertools import chain, zip_longest
import pickle
import re
from sklearn.model_selection import train_test_split

In [83]:
train_replace_dict = {'atis_flight#atis_airfare': 'atis_airfare', 
                                     'atis_ground_service#atis_ground_fare': 'atis_ground_fare',
                                     'atis_aircraft#atis_flight#atis_flight_no': 'atis_flight_no',
                                     'atis_airfare#atis_flight_time':'atis_flight_time',
                                     'atis_airline#atis_flight_no':'atis_flight_no',
                                     'atis_flight_no#atis_airline' : 'atis_flight_no',
                                     'atis_airfare#atis_flight': 'atis_airfare',
                                     'atis_flight#atis_airline' : 'atis_airline'}

# intent to idx mapping
intent_list = (pd.read_csv('../../data/multiATIS/intent_list.csv',sep=',',header=None,names=['INTENT']).INTENT.values.tolist())
intent2idx = { intent:idx for idx,intent in enumerate(intent_list)}

# slot idx mapping
final_slots = pd.read_csv('../../data/multiATIS/slots_list.csv',sep=',',header=None,names=['SLOTS']).SLOTS.values.tolist()
slots2idx  = {slots:idx for idx,slots in enumerate(final_slots)}
idx2slots = {v: k for k, v in slots2idx.items()}

In [84]:
with open('../../data/BG_Noise_Phrase.txt') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
phrase = [x.strip() for x in content] 

In [85]:
def mapping2idx(df):
    
    #del df['id']

    # adding intent encoding
    encoded_intent = []
    for intent in list(df.INTENT):
        try:
            encoded_intent.append(intent2idx[intent])
        except:
            print(intent)
            encoded_intent.append(intent2idx['OOI'])
            
        
    df['INTENT_ID'] = encoded_intent 

    # adding mapped slot column
    slots_ID = []

    for annotations in list(df.SLOTS):
        encoded_slot = ""
        #print((annotations.split()))
        for tokens in annotations.split():
            try: 
                encoded_slot += str(slots2idx[tokens]) + " "
            except:
                print('token',tokens)
                encoded_slot += str(slots2idx['O']) + " "
        slots_ID.append(" ".join(encoded_slot.split()))

    df['SLOTS_ID'] = slots_ID
    
    return df

In [86]:
def get_carrier_phrase_length(annotations):
    CP_idx = []
    for idx,token in enumerate(annotations.split(' ')):
        if token == 'O':
            CP_idx.append(idx)
    
    return CP_idx 

def twolists(l1, l2):
    return [x for x in chain.from_iterable(zip_longest(l1, l2)) if x is not None]

In [87]:
def BG_Noise(data):
    
    orig_data = deepcopy(data)
    
    augINTENT, augSLOTS, augTEXT,augID = [],[],[],[]
    
    cnt = 0
    for sample in data.values.tolist():
        
        CP_idx = get_carrier_phrase_length(sample[2])
        
        CP_length = len(CP_idx)
        
        TEXT, SLOTS = sample[1], sample[2]
        
        if CP_length >= 20:
            del_index = 20
            TEXT = sample[1][:del_index]
            SLOTS = sample[2][:del_index]
        
        bg_TEXT = random.sample(phrase,1)[0]
        bg_SLOTS = re.sub(' +', ' ', 'O '*len(bg_TEXT.split(' ')))
        
        noisyTEXT = twolists(TEXT.split(' '), bg_TEXT.split(' '))
        noisySLOTS = twolists(SLOTS.split(' '), bg_SLOTS.split(' '))
        noisySLOTS = re.sub(' +', ' ',' '.join(noisySLOTS))
        #print(len(bg_TEXT.split(' ')), len(bg_SLOTS.split()))
        #print(len(TEXT.split(' ')), len(SLOTS.split()))
        #print(len(noisyTEXT), len(noisySLOTS))
        
        
        #assert(len(noisySLOTS)==len(noisyTEXT))
        augINTENT.append(sample[3])
        augTEXT.append(' '.join(noisyTEXT))
        augSLOTS.append(noisySLOTS)
        augID.append(cnt)
        cnt+=1
        
        #print(' '.join(noisyTEXT),noisySLOTS)
               
    augPD = pd.DataFrame([augID,augTEXT,augSLOTS,augINTENT],index=['ID','TEXT','SLOTS','INTENT']).T
    
    return augPD
    

In [88]:
def ASRN_aug(data_path,out_path,fn):
    
    # loading dataset
    data = pd.read_csv(data_path,sep='\t',header=0,names=['ID','TEXT','SLOTS','INTENT'])
    
    # lowerCasing the TEXT column
    data['TEXT'] = data['TEXT'].str.lower()
    
    # handling multi-label instances 
    data['INTENT'] = data['INTENT'].replace(train_replace_dict)
    
    aug_data = BG_Noise(data)
    aug_data = mapping2idx(aug_data)
        
    aug_data = aug_data.sample(frac=1).reset_index(drop=True)
    aug_data.to_csv(out_path+fn,sep='\t',index=False)
    

In [89]:
ASRN_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/BG_Noise/','test_EN_01.tsv')
ASRN_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/BG_Noise/','test_EN_02.tsv')
ASRN_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/BG_Noise/','test_EN_03.tsv')
ASRN_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/BG_Noise/','test_EN_04.tsv')
ASRN_aug('../../data/multiATIS/raw/train_dev_test/test_EN.tsv','../../data/multiATIS/split/test/BG_Noise/','test_EN_05.tsv')

atis_day_name
atis_day_name
token B-fromloc.
token B-compartment
token B-fromloc.ci
token B-stoploc.airport_code
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token B-fromloc.
token B-compartment
token B-fromloc.ci
token B-stoploc.airport_code
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token B-fromloc.
token B-compartment
token B-fromloc.ci
token B-stoploc.airport_code
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token B-fromloc.
token B-compartment
token B-fromloc.ci
token B-stoploc.airport_code
token B-booking_class
token B-flight
atis_day_name
atis_day_name
token B-fromloc.
token B-compartment
token B-fromloc.ci
token B-stoploc.airport_code
token B-booking_class
token B-flight
